In [1]:
# set random seed
import random
# random.seed(42)
import numpy as np
# np.random.seed(42)
import torch
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# Parameters

In [2]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.5 # polynomial degree
e = 0.5 # noise width

In [6]:
seed_all = np.arange(1,3)
cost_Oracle_all = {}; cost_SPO_all = {}; cost_OLS_all = {}; cost_DDR_all = {}

for seed in seed_all:
    from Data import data_generation
    data_gen = data_generation()

    #  ****** Data generation *********
    feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=seed)
    # split train test data
    from sklearn.model_selection import train_test_split
    x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

    #  ****** SPO *********
    print("*** seed = ",seed,": Run SPO ========")
    from SPO_Plus import run_SPO_Shortest_Path
    SPO_runner = run_SPO_Shortest_Path()
    batch_size = 20
    num_epochs = 30
    arcs,loader_train,loader_test,cost_Oracle_all[seed],cost_SPO_all[seed] = SPO_runner.run(x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)

    #  ****** OLS *********
    print("*** seed = ",seed,": Run OLS ========")
    from OLS import run_OLS_Shortest_Path
    OLS_runner = run_OLS_Shortest_Path()
    cost_OLS_all[seed] = OLS_runner.run(arcs,x_train,c_train,grid,loader_test)
    # print("Average OLS Cost = ",np.mean(cost_OLS))


    #  ****** DDR *********
    print("*** seed = ",seed,": Run DDR ========")
    from DDR import run_DDR_Shortest_Path
    DDR_runner = run_DDR_Shortest_Path()
    mu_arr = np.arange(0.1,1,0.05)
    lamb_arr = np.arange(0.1,1,0.05)
    lamb_arr = [0.1]
    minimum_value = 1000000000
    for lamb in lamb_arr:
        # print("======== lambda = ",lamb,"============")
        for mu in mu_arr:
            num_nodes = 25
            # w0_ddr_val,W_ddr_val = solve_DDR(lamb,mu,num_nodes,x_train,c_train)
            cost_DDR = DDR_runner.run(arcs,x_train, c_train, grid,loader_test,lamb,mu,num_nodes)
            cost_DDR_all[seed,lamb,mu] = cost_DDR
            # if np.mean(cost_DDR) < minimum_value:
            #     minimum_value = np.mean(cost_DDR)
            #     print("lambda = ",lamb, ", mu = ",mu, ", Lowest Average DDR Cost = ",minimum_value, " Std = ",np.std(cost_DDR))
            # print("lambda = ",lamb, ", mu = ",mu, ",Average DDR Cost = ",np.mean(cost_DDR), " Std = ",np.std(cost_DDR))

*** seed =  1 : Run SPO ========
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25
Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1808.48it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2084.34it/s]


Num of cores: 2
*** seed =  1 : Run OLS ========
*** seed =  1 : Run DDR ========
*** seed =  2 : Run SPO ========
Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1384.01it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2083.78it/s]


Num of cores: 2
*** seed =  2 : Run OLS ========
*** seed =  2 : Run DDR ========


# Results

In [13]:
seed = 2
lamb = 0.1

print("Seed = ",seed,", Average Oracle Cost = ",np.round(np.mean(cost_Oracle_all[seed]),4),"Std = ", np.round(np.std(cost_Oracle_all[seed]),4))
print("Seed = ",seed,", Average SPO Cost = ", np.round(np.mean(cost_SPO_all[seed]),4),"Std = ", np.round(np.std(cost_SPO_all[seed]),4))
print("Seed = ",seed,", Average OLS Cost = ", np.round(np.mean(cost_OLS_all[seed]),4),"Std = ", np.round(np.std(cost_OLS_all[seed]),4))
for lamb in lamb_arr:
    # print("======== lambda = ",lamb,"============")
    for mu in mu_arr:
        print("Seed = ",seed,", lamb = ",lamb,",mu = ",mu,", Average DRR Cost = ", np.round(np.mean(cost_DDR_all[seed,lamb,mu]),4),"Std = ", np.round(np.std(cost_DDR_all[seed,lamb,mu]),4))

Seed =  2 , Average Oracle Cost =  6.0116 Std =  1.2782
Seed =  2 , Average SPO Cost =  6.9605 Std =  1.582
Seed =  2 , Average OLS Cost =  6.9325 Std =  1.595
Seed =  2 , lamb =  0.1 ,mu =  0.1 , Average DRR Cost =  6.924 Std =  1.5888
Seed =  2 , lamb =  0.1 ,mu =  0.15000000000000002 , Average DRR Cost =  6.9298 Std =  1.6025
Seed =  2 , lamb =  0.1 ,mu =  0.20000000000000004 , Average DRR Cost =  6.9328 Std =  1.6049
Seed =  2 , lamb =  0.1 ,mu =  0.25000000000000006 , Average DRR Cost =  6.929 Std =  1.597
Seed =  2 , lamb =  0.1 ,mu =  0.30000000000000004 , Average DRR Cost =  6.9268 Std =  1.5945
Seed =  2 , lamb =  0.1 ,mu =  0.3500000000000001 , Average DRR Cost =  6.933 Std =  1.6003
Seed =  2 , lamb =  0.1 ,mu =  0.40000000000000013 , Average DRR Cost =  6.9348 Std =  1.6041
Seed =  2 , lamb =  0.1 ,mu =  0.4500000000000001 , Average DRR Cost =  6.9332 Std =  1.6003
Seed =  2 , lamb =  0.1 ,mu =  0.5000000000000001 , Average DRR Cost =  6.9392 Std =  1.6059
Seed =  2 , lamb 